In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# ----------------------------
# CONFIGURACIÓN
# ----------------------------
IM_SIZE = 64

# ----------------------------
# CARGAR MODELO Y PCA
# ----------------------------
model = load_model("sign_model.h5")
pca_components = np.load("pca_components.npy")
pca_mean = np.load("pca_mean.npy")
classes = np.load("label_classes.npy", allow_pickle=True)

# ----------------------------
# APLICAR PCA
# ----------------------------
def apply_pca(flat_image):
    return (flat_image - pca_mean) @ pca_components.T

# ----------------------------
# CARGAR Y PROCESAR IMAGEN
# ----------------------------
def preprocess_image(path):
    img = cv2.imread(path)

    if img is None:
        raise ValueError(f"No se pudo abrir la imagen: {path}")

    img = cv2.resize(img, (IM_SIZE, IM_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.astype("float32") / 255.0

    flat = img.flatten()

    # reducir usando PCA
    pca_vec = apply_pca(flat)

    return pca_vec.reshape(1, -1)

# ----------------------------
# PREDICCIÓN
# ----------------------------
def predict(path):
    vec = preprocess_image(path)
    pred = model.predict(vec)
    idx = np.argmax(pred)
    letter = classes[idx]
    confidence = pred[0][idx]

    print("----------------------------")
    print(f"Imagen: {path}")
    print(f"Letra predicha: {letter}")
    print(f"Confianza: {confidence:.4f}")
    print("----------------------------")

    return letter


# ----------------------------
# EJEMPLO DE USO
# ----------------------------
if __name__ == "__main__":
    test_image = "Image/E/15.png"  
     # cambia esto por la imagen a probar
    predict(test_image)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
----------------------------
Imagen: Image/Z/15.png
Letra predicha: Z
Confianza: 0.9972
----------------------------


## Random Forest

In [12]:
import numpy as np
import joblib

# Cargar modelos
rf = joblib.load("rf_model.pkl")
scaler = joblib.load("rf_scaler.pkl")

pca_components = np.load("rf_pca_components.npy")
pca_mean = np.load("rf_pca_mean.npy")
classes = np.load("rf_label_classes.npy", allow_pickle=True)

# PCA manual
def apply_pca(vec):
    return (vec - pca_mean) @ pca_components.T


def predict(values_dict):
    # Convertir dict → vector
    keys = [
        "x","y","z",
        "thumb","fore","index","ring","little",
        "thumb2","fore2","index2","ring2","little2",
        "keycode","gs1","gs2","sign",
        "giroX","giroY","giroZ"
    ]

    vec = np.array([values_dict[k] for k in keys], dtype="float32")

    # Normalizar
    vec = scaler.transform([vec])[0]

    # PCA
    vec_pca = apply_pca(vec)

    # Predicción
    idx = rf.predict([vec_pca])[0]
    return classes[idx]


In [13]:
pred = predict({
    "x": -120, "y": 80, "z": 60,
    "thumb": 0,"fore":0,"index":0,"ring":0,"little":0,
    "thumb2":20,"fore2":100,"index2":100,"ring2":100,"little2":100,
    "keycode":0,"gs1":0,"gs2":1,"sign":0,
    "giroX":0,"giroY":0,"giroZ":0
})

print("Letra predicha:", pred)


Letra predicha: A


## SVM

df = pd.read_csv("dataset.csv")

fila = df.iloc[100]  # ejemplo

pred = svm_predict_from_row(fila)
print("Predicción:", pred)


In [14]:
import joblib
import numpy as np
import cv2



def svm_predict_from_image(img_path):
    import cv2
    import numpy as np
    import joblib

    # Cargar modelos
    svm = joblib.load("svm_model.pkl")
    scaler = joblib.load("svm_scaler.pkl")
    pca = joblib.load("svm_pca.pkl")
    label_classes = np.load("svm_label_classes.npy", allow_pickle=True)

    # Procesar imagen
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64, 64))
    img = img.flatten() / 255.0

    # Número de features que se usaron en el entrenamiento
    n_features_model = scaler.n_features_in_

    # Calculamos cuántas features numéricas necesita el dummy
    n_csv_features = n_features_model - len(img)

    csv_dummy = np.zeros((n_csv_features,), dtype=float)

    X = np.hstack([img, csv_dummy]).reshape(1, -1)

    # Pasar por scaler y PCA
    X_scaled = scaler.transform(X)
    X_pca = pca.transform(X_scaled)

    # Predicción
    pred_idx = svm.predict(X_pca)[0]
    pred_label = label_classes[pred_idx]

    print("Predicción:", pred_label)
    return pred_label




In [ ]:
svm_predict_from_image("Image/E/15.png")

Predicción: I


'I'

In [16]:
import pandas as pd

# Métricas de cada modelo (ajusta estos valores con los que realmente obtuviste)
metrics = {
    "Modelo": ["Red Neuronal", "Random Forest", "SVM"],
    "Accuracy": [
        0.9867,   # ejemplo: accuracy validación Red Neuronal
        0.9962,   # accuracy validación Random Forest (del print)
        0.9954    # accuracy SVM (del print)
    ],
    "Precision": [
        0.9860,   # pon aquí la precisión de la red si la calculas
        0.9960,   # si calculas precision para RF
        0.9956    # precision SVM
    ],
    "Recall": [
        0.9860,   # recall red
        0.9960,   # recall RF
        0.9954    # recall SVM
    ],
    "F1-Score": [
        0.9860,   # F1 red
        0.9960,   # F1 RF
        0.9954    # F1 SVM
    ]
}

df_metrics = pd.DataFrame(metrics)
display(df_metrics)

# Mostrar también cuál es el mejor modelo por cada métrica
best_accuracy_model = df_metrics.loc[df_metrics["Accuracy"].idxmax(), "Modelo"]
best_precision_model = df_metrics.loc[df_metrics["Precision"].idxmax(), "Modelo"]
best_recall_model = df_metrics.loc[df_metrics["Recall"].idxmax(), "Modelo"]
best_f1_model = df_metrics.loc[df_metrics["F1-Score"].idxmax(), "Modelo"]

print("Mejor modelo en Accuracy:", best_accuracy_model)
print("Mejor modelo en Precision:", best_precision_model)
print("Mejor modelo en Recall:", best_recall_model)
print("Mejor modelo en F1-Score:", best_f1_model)


,Modelo,Accuracy,Precision,Recall,F1-Score
0,Red Neuronal,0.9867,0.9860,0.9860,0.9860
1,Random Forest,0.9962,0.9960,0.9960,0.9960
2,SVM,0.9954,0.9956,0.9954,0.9954


Mejor modelo en Accuracy: Random Forest
Mejor modelo en Precision: Random Forest
Mejor modelo en Recall: Random Forest
Mejor modelo en F1-Score: Random Forest
